In [3]:
import yfinance as yf
import pandas as pd
import numpy as np


In [13]:
def get_options_chain(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    
    # Get available expiration dates
    expiration_dates = ticker.options
    options_chain = {}
    
    for expiration_date in expiration_dates:
        options_chain_data = ticker.option_chain(expiration_date)
        options_chain[expiration_date] = {
            'calls': options_chain_data.calls,
            'puts': options_chain_data.puts
        }
    
    return options_chain

# ticker_symbol = "AAPL" 
# options_chain = get_options_chain(ticker_symbol)

# first_expiration_date = list(options_chain.keys())[0]
# print(f"Options chain for {ticker_symbol} on {first_expiration_date}:")
# print("Calls:")
# print(options_chain[first_expiration_date]['calls'])
# print("\nPuts:")
# print(options_chain[first_expiration_date]['puts'])


In [20]:
data=pd.read_excel("Stock List.xlsx")
remove_stock=['LSDI', 'EVBG', 'AKLI', 'WIRE', 'PWFL', 'VERY', 'CVLY', 'TNP']
data=data[~data['Ticker'].isin(remove_stock)] 
All_Tickers=data.Ticker.to_list()
All_Tickers = [ticker for ticker in All_Tickers if isinstance(ticker, str)]

In [24]:
All_Tickers[:4000]

['AAPL',
 'MSFT',
 'NVDA',
 'GOOG',
 'AMZN',
 'META',
 'TSM',
 'LLY',
 'TSLA',
 'AVGO',
 'JPM',
 'WMT',
 'NVO',
 'UNH',
 'V',
 'XOM',
 'MA',
 'PG',
 'JNJ',
 'ORCL',
 'COST',
 'HD',
 'ABBV',
 'ASML',
 'BAC',
 'KO',
 'MRK',
 'NFLX',
 'AZN',
 'CVX',
 'ADBE',
 'SAP',
 'PEP',
 'TM',
 'NVS',
 'CRM',
 'TMO',
 'TMUS',
 'AMD',
 'LIN',
 'ACN',
 'SHEL',
 'MCD',
 'CSCO',
 'BABA',
 'ABT',
 'PM',
 'WFC',
 'DHR',
 'IBM',
 'FMX',
 'TXN',
 'QCOM',
 'AXP',
 'GE',
 'AMGN',
 'INTU',
 'NOW',
 'RY',
 'ISRG',
 'NEE',
 'UL',
 'SPGI',
 'CAT',
 'PFE',
 'DIS',
 'HSBC',
 'MS',
 'RTX',
 'TTE',
 'HDB',
 'UNP',
 'GS',
 'CMCSA',
 'UBER',
 'AMAT',
 'T',
 'SNY',
 'PGR',
 'LOW',
 'SYK',
 'LMT',
 'BHP',
 'TJX',
 'HON',
 'BLK',
 'ARM',
 'PDD',
 'REGN',
 'BKNG',
 'ELV',
 'COP',
 'BUD',
 'MUFG',
 'NKE',
 'VRTX',
 'BSX',
 'PLD',
 'SCHW',
 'CB',
 'C',
 'MDT',
 'SONY',
 'ETN',
 'MMC',
 'PANW',
 'ADP',
 'ADI',
 'UPS',
 'AMT',
 'KKR',
 'TD',
 'DE',
 'SBUX',
 'IBN',
 'MELI',
 'ANET',
 'HCA',
 'BMY',
 'MDLZ',
 'BX',
 'BA',
 'MU',


In [26]:
prices_data = yf.download(All_Tickers[:4000], start='2024-01-01')


[*********************100%%**********************]  4000 of 4000 completed


In [28]:
prices_data.keys()

MultiIndex([('Adj Close',    'A'),
            ('Adj Close',   'AA'),
            ('Adj Close', 'AACG'),
            ('Adj Close', 'AADI'),
            ('Adj Close',  'AAL'),
            ('Adj Close',  'AAN'),
            ('Adj Close', 'AAOI'),
            ('Adj Close', 'AAON'),
            ('Adj Close',  'AAP'),
            ('Adj Close', 'AAPL'),
            ...
            (   'Volume',  'ZTO'),
            (   'Volume',  'ZTS'),
            (   'Volume', 'ZUMZ'),
            (   'Volume',  'ZUO'),
            (   'Volume', 'ZURA'),
            (   'Volume', 'ZVIA'),
            (   'Volume', 'ZVRA'),
            (   'Volume',  'ZWS'),
            (   'Volume', 'ZYME'),
            (   'Volume', 'ZYXI')],
           names=['Price', 'Ticker'], length=24000)

In [117]:
def gkyz_volatility(data):
    """
    Calculate the Garman-Klass-Yang-Zhang (GKYZ) volatility estimator.

    Parameters:
    - data: A DataFrame containing the columns 'Open', 'High', 'Low', and 'Close'.

    Returns:
    - GKYZ Volatility (annualized).
    """
    
    # Calculate the log returns
    log_high_low = np.log(data['High'] / data['Low'])**2
    log_close_open = np.log(data['Adj Close'] / data['Open'])**2
    log_close_yesterday = np.log(data['Adj Close'] / data['Adj Close'].shift(1))**2

    sigma_hl = (log_high_low / (4 * np.log(2))).mean()
    sigma_oc = (log_close_open * 0.5).mean()
    sigma_cc = log_close_yesterday.mean()

    # GKYZ Volatility calculation
    gkyz_var = 0.5 * sigma_hl + (2 * np.log(2) - 1) * sigma_cc + sigma_oc
    gkyz_volatility = np.sqrt(gkyz_var)

    # Annualize the volatility assuming 252 trading days per year
    annualized_volatility = gkyz_volatility * np.sqrt(252)
    
    return annualized_volatility

In [129]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a given data set.

    Parameters:
    - data: A DataFrame or Series of closing prices.
    - window: The number of periods to use in the calculation (default is 14).

    Returns:
    - A Pandas Series representing the RSI values.
    """
    
    # Calculate the price differences
    delta = data.diff()
    
    # Separate the gains and losses
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    
    # Calculate the rolling averages
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    
    # Calculate the RS (Relative Strength)
    rs = avg_gain / avg_loss
    
    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    
    return pd.Series(rsi, name='RSI')

In [131]:
volatility_results = {}

# Iterate over each ticker's data
for ticker in prices_data.columns.levels[1]:
    # Extract the data for the current ticker
    ticker_data = prices_data.xs(ticker, level='Ticker', axis=1).dropna()
    total_change=ticker_data['Adj Close'][-1]/ ticker_data['Adj Close'].iloc[0]
    rsi=calculate_rsi(ticker_data['Adj Close'], window=14)
    if total_change>0.5 and ticker_data[-30:]['Volume'].mean()>=1000000 and rsi[-1]>60:
        volatility = gkyz_volatility(ticker_data)
        volatility_results[ticker] = volatility
    else:
        pass
        # Store the result

# Convert the results to a DataFrame for easier viewing
volatility_df = pd.DataFrame.from_dict(volatility_results, orient='index', columns=['GKYZ Volatility'])

# Display the results
print(volatility_df)

C:\Users\adnan\AppData\Local\Temp\ipykernel_13316\2044135339.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_change=ticker_data['Adj Close'][-1]/ ticker_data['Adj Close'].iloc[0]


KeyError: -1

In [119]:
volatility_df.sort_values(by='GKYZ Volatility',ascending=False)

,GKYZ Volatility
NNE,2.575076
RNLX,2.484754
SMMT,2.123508
BOF,2.057790
SOND,2.055713
...,...
EQC,0.145554
NVS,0.144980
KO,0.140182
UL,0.138275


In [121]:
volatility_df[volatility_df.index=='AAPL']

,GKYZ Volatility
AAPL,0.250549


In [132]:
test=calculate_rsi(AAPL['Adj Close'])